In [ ]:
#https://www.dataquest.io/blog/python-excel-xlwings-tutorial/

In [ ]:
import pandas as pd
import xlwings as xw
df = pd.read_csv(r'C:\Users\jaw\OneDrive - Mountain G. Enterprises, Inc\Documents\Programming\xlwings\Dataquest_Tutorial\euromillions.csv')
df.sample(5)

In [ ]:
wb = xw.Book() # wb = xw.Book(filename) would open an existing file

In [ ]:
print(xw.apps)

In [ ]:
print(wb.sheets)

In [ ]:
#creates a worksheet object assigns it to ws
ws = wb.sheets["Sheet1"]
#checks that wb.sheets[0] equals ws
print(ws == wb.sheets[0])

In [ ]:
ws.name = "EuroMillions"

In [ ]:
#ws.range("A1") is a Range object
ws.range("A1").value = df

In [ ]:
ws.clear_contents()

In [ ]:
ws.range("A1").options(index=False).value = df

In [ ]:
last_row = ws.range(1,1).end('down').row
print("The last row is {row}.".format(row=last_row))
print("The DataFrame df has {rows} rows.".format(rows=df.shape[0]))

In [ ]:
ws.range(
"A2:N{row}".format(row=last_row)
).api.Sort(Key1=ws.range("A:A").api, Order1=1)

In [ ]:
ws.range("O1").value = "Date"

In [ ]:
ws.range("O2").value = "=C2&D2&RIGHT(E2, 2)"

In [ ]:
ws.range("O2").api.AutoFill(ws.range("O2:O{row}".format(row=last_row)).api,0)

In [ ]:
def autofill(worksheet, cell, last_row):
   rg_cell = worksheet.range(cell)
   to_fill = "{col}{top_row}:{col}{last_row}".format(
      col=rg_cell.get_address(0,0)[0],
      top_row=rg_cell.row,
      last_row=last_row
   )
   rg_cell.api.Autofill(worksheet.range(to_fill).api, 0)

In [ ]:
print(type(ws.range("O2:O{row}".format(row=last_row)).value))

In [ ]:
print(ws.range('O2:O{row}'.format(row=last_row)).value[:10])

In [ ]:
ws.range('O2').options(transpose=True).value = ws.range('O2:O{row}'.format(row=last_row)).value

In [ ]:
ws.range('C:E').api.Delete()

In [ ]:
ws.range('1:1').api.Font.Bold = True

In [ ]:
ws.range('J:J').number_format = "$#,##0"

In [ ]:
last_column = ws.range(1,1).end('right').get_address(0,0)[0]

In [ ]:
print(last_column)

In [ ]:
ws.range('A1:{}1'.format(last_column)).api.Borders(9).LineStyle = -4119

In [ ]:
ws.autofit()

In [ ]:
ws.range('A:L').api.ColumnWidth = ws.range('J:J').api.ColumnWidth

In [ ]:
wb.sheets.add('Frequencies')

In [ ]:
frequencies = wb.sheets['Frequencies']

In [ ]:
# add a header for the numbers
frequencies.range('A1').value = 'Number'

In [ ]:
# populate the fifty cells immediately below with the numbers 1 through 50
# since we're starting from the second row, we'll want to substract 1 from the row
frequencies.range('A2:A51').value = '=ROW()-1'

In [ ]:
# add a header for the frequencies
frequencies.range('B1').value = 'Frequency'

In [ ]:
# insert on B2 the result of a standard Excel formula
frequencies.range('B2').value = '=COUNTIF(Euromillions!$C$2:$G$201,Frequencies!A2)'

In [ ]:
autofill(frequencies, 'B2', 51)

In [ ]:
frequencies.range('D1').value = 'Lucky Star'

In [ ]:
frequencies.range('E1').value = 'Frequency'

In [ ]:
frequencies.range('D2:D13').value = '=ROW()-1'

In [ ]:
frequencies.range('E2').value ='=COUNTIF(EuroMillions!$H$2:$I$201,Frequencies!D2)'

In [ ]:
autofill(frequencies, 'E2', 13)

In [ ]:
frequencies.autofit()

In [ ]:
wb.sheets.add('Graphs')

In [ ]:
nr_freq = xw.Chart()

In [ ]:
nr_freq.name = 'Number Frequencies'

In [ ]:
nr_freq.set_source_data(frequencies.range('Frequencies!B1:B51'))

In [ ]:
nr_freq.api[1].FullSeriesCollection(1).XValues = '=Frequencies!A2:A51'

In [ ]:
nr_freq.chart_type = 'column_clustered'

In [ ]:
nr_freq.height = 250
nr_freq.width = 750

In [ ]:
nr_freq.api[1].SetElement(2)  # Place chart title at the top
nr_freq.api[1].ChartTitle.Text = 'Number Frequencies'

In [ ]:
nr_freq.api[1].HasLegend = 0

In [ ]:
nr_freq.api[1].Axes(1).TickLabelSpacing = 1

In [ ]:
ls_freq = xw.Chart()
ls_freq.top = 250
ls_freq.name = 'Lucky Star Frequencies'
ls_freq.set_source_data(frequencies.range('Frequencies!E1:E13'))
ls_freq.api[1].FullSeriesCollection(1).XValues = '=Frequencies!D2:D13'
ls_freq.chart_type = 'column_clustered'
ls_freq.height = 250
ls_freq.width = 750
ls_freq.api[1].SetElement(2)
ls_freq.api[1].ChartTitle.Text = 'Lucky Star Frequencies'
ls_freq.api[1].HasLegend = 0
ls_freq.api[1].Axes(1).TickLabelSpacing = 1

In [ ]:
jackpot = xw.Chart()
jackpot.top = 500
jackpot.name = 'Jackpot'
last_row = ws.range(1,1).end('down').row
jackpot.set_source_data(ws.range('Euromillions!J2:J{}'.format(last_row)))
jackpot.api[1].FullSeriesCollection(1).XValues\
= 'Euromillions!L2:L{}'.format(last_row)
jackpot.chart_type = 'line'
jackpot.height = 250
jackpot.width = 750
jackpot.api[1].SetElement(2)
jackpot.api[1].ChartTitle.Text = 'Jackpot'
jackpot.api[1].HasLegend = 0

In [ ]:
wb.save('EuroMillions.xlsx')